In [391]:
# pip install scikit-learn
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.spatial import distance
import warnings

import psycopg2                                     # Для подключение к БД

host = '**.**.***.***'
database = 'sas'
dbuser = '********'
dbpassword = '********'

greenplum = psycopg2.connect(host = host, database = database, user = dbuser, password = dbpassword)
gp_cursor = greenplum.cursor()
warnings.filterwarnings('ignore')

In [392]:
query = """
select
    object_bk
    ,object_format_desc_en
	,case 	when region_name_en like '%MOSCOW%' then 'MSK'
			when region_name_en like '%ST-PET%' then 'SPB'
			else 'Regions'
			end as region
	,count(distinct id) 			as "checks"
	,count(distinct accountid) 		as "clients"
	,round(count(distinct id)::decimal/count(distinct accountid),2) as "frequency"
	,round(avg(totalvalue/100),2) 	as "avg_check"
	,median(totalvalue/100) 		as "median_check"
from ods_mdm.dim_stores_bi bi
left join ods_clm.fct_transactions tr on bi.object_bk = tr.locationcode
where object_close_dt is null and type='ER' and status='B' and date::date >= current_date - interval '6 month'
group by 1,2,3
order by 1;
"""

In [522]:
df  =  pd.read_sql(query, greenplum)
df.head()

,object_bk,object_format_desc_en,region,checks,clients,frequency,avg_check,median_check
0,1,Auchan Hyper,MSK,830283,151413,5.48,2598.25,1507.99
1,101,Auchan Hyper,SPB,571579,109252,5.23,1700.38,999.50
2,102,Auchan Hyper,SPB,465048,88920,5.23,2354.95,1434.53
3,14,Auchan Hyper,MSK,567983,134274,4.23,2226.90,1222.73
4,15,Auchan Hyper,MSK,1297411,177230,7.32,1995.95,1045.74


In [523]:
# Собираем словарь с данными по магазину
info = {}

for i in df['object_bk'].unique():
    obj = str(df[df['object_bk']==str(i)]['object_format_desc_en']).split('\n')[0] .split('    ')[1]
    reg = str(df[df['object_bk']==str(i)]['region']).split('\n')[0] .split('    ')[1]
    info[i] = (obj, reg)

# Меняем тип столбца
df['object_bk'] = df['object_bk'].astype(int)
df.head()

,object_bk,object_format_desc_en,region,checks,clients,frequency,avg_check,median_check
0,1,Auchan Hyper,MSK,830283,151413,5.48,2598.25,1507.99
1,101,Auchan Hyper,SPB,571579,109252,5.23,1700.38,999.50
2,102,Auchan Hyper,SPB,465048,88920,5.23,2354.95,1434.53
3,14,Auchan Hyper,MSK,567983,134274,4.23,2226.90,1222.73
4,15,Auchan Hyper,MSK,1297411,177230,7.32,1995.95,1045.74


In [524]:
def search_nearest(shop_num):
    """
    Функция для подбора наиболее подходящего магазина.
    Принимает на вход намер магазина.
    Возвращает список наиболее подходящих магазинов.
    
    При необходимости в metrix необходимо указать другие метрики
    """
    dict = {}
    # указываем метрики, которые будем сравнивать
    metrix = ['checks','clients', 'frequency', 'avg_check', 'median_check']

    base = df[df['object_bk'] == shop_num][metrix]
    obj = str(df[df['object_bk']==shop_num]['object_format_desc_en']).split('\n')[0].split('    ')[1]
    reg = str(df[df['object_bk']==shop_num]['region']).split('\n')[0].split('    ')[1]

    same_shops = df[(df['object_format_desc_en']==obj) & (df['region']==reg)]

    for i in same_shops['object_bk'].unique():
        if i == shop_num:
            pass
        else:
            compare = same_shops[same_shops['object_bk'] == i][metrix]
            dist = distance.cdist(base, compare, 'euclidean')

            dict[i] = round(dist[0][0],2)

    min_dist_shop = min(dict, key=dict.get)
    min_dist = min(dict.values())
    print('Номер магазина, наиболее похожего на магазин {} {}: магазин № {} {}. Растояние: {}'.format(shop_num, info[str(shop_num)], min_dist_shop, info[str(min_dist_shop)], min_dist))

    return sorted(dict.items(), key=lambda item: item[1])

In [525]:
search_nearest(41)

Номер магазина, наиболее похожего на магазин 41 ('Auchan Hyper', 'Regions'): магазин № 16 ('Auchan Hyper', 'Regions'). Растояние: 4581.43


[(16, 4581.43),
 (28, 18192.91),
 (201, 33456.59),
 (21, 56012.15),
 (39, 57596.14),
 (38, 86090.04),
 (19, 108596.59),
 (42, 108994.59),
 (18, 110244.04),
 (63, 117483.16),
 (43, 120021.5),
 (57, 130498.21),
 (64, 132075.63),
 (62, 138112.68),
 (23, 149366.17),
 (33, 154570.84),
 (54, 165324.83),
 (34, 175042.82),
 (17, 187399.86),
 (301, 190340.48),
 (37, 202744.4),
 (56, 205927.26),
 (26, 221733.48),
 (729, 222626.22),
 (22, 224139.66),
 (67, 251249.12),
 (59, 262724.58),
 (61, 277246.02),
 (65, 290502.13),
 (25, 301053.44),
 (46, 301355.69),
 (69, 431068.61),
 (73, 488073.64)]